In [1]:
import pandas as pd
from scipy.sparse import csr_matrix

def load_data(movies_file, ratings_file):
    movies = pd.read_csv(movies_file)
    ratings = pd.read_csv(ratings_file)
    
    # Create the user-item matrix
    user_item_matrix = ratings.pivot(index='User_ID', columns='Movie_ID', values='Rating')
    # Fill missing values (here we fill with 0)
    user_item_matrix.fillna(0, inplace=True)
    sparse_user_item = csr_matrix(user_item_matrix.values)
    return movies, ratings, user_item_matrix, sparse_user_item


### Module 2: Factor Model Training and Bias Computation

In [2]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

def train_factor_model(sparse_user_item, n_components=30):
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    user_factors = svd.fit_transform(sparse_user_item)  # shape: (num_users, n_components)
    movie_factors = svd.components_.T                    # shape: (num_movies, n_components)
    return user_factors, movie_factors, svd

def compute_biases(user_item_matrix):
    # Global bias: mean of all nonzero ratings.
    all_ratings = user_item_matrix.values[user_item_matrix.values != 0]
    global_bias = all_ratings.mean()
    # Movie biases: average deviation per movie (only on nonzero ratings)
    movie_biases = user_item_matrix.replace(0, pd.NA).mean() - global_bias
    # User biases (if needed)
    user_biases = user_item_matrix.replace(0, pd.NA).mean(axis=1) - global_bias
    return global_bias, movie_biases, user_biases


In [3]:
from scipy.stats import norm

def phi(x):
    """Standard normal PDF."""
    return norm.pdf(x)

def Phi(x):
    """Standard normal CDF."""
    return norm.cdf(x)

### Module 4: Pairwise Preference Model & Information Gain

In [4]:
def compute_delta(mu, Sigma, v_l, v_r, b_il, b_ir):
    """
    Compute:
      delta = (b_il - b_ir + (v_l - v_r)^T mu) / s_lr
    where s_lr = sqrt( v_l^T Sigma v_l + v_r^T Sigma v_r ).
    """
    v_diff = v_l - v_r
    num = (b_il - b_ir) + np.dot(v_diff, mu)
    s_lr = np.sqrt(np.dot(v_l, np.dot(Sigma, v_l)) + np.dot(v_r, np.dot(Sigma, v_r)) + 1e-8)
    return num / s_lr, s_lr, v_diff

def information_gain(mu, Sigma, v_l, v_r, b_il, b_ir):
    """
    Computes the IG criterion:
      IG = log( 1 + (1/s_lr^2) * (phi(delta)^2/(Phi(delta)(1-Phi(delta))) *
             (v_l - v_r)^T Sigma (v_l - v_r) ) )
    """
    delta, s_lr, v_diff = compute_delta(mu, Sigma, v_l, v_r, b_il, b_ir)
    Phi_delta = np.clip(Phi(delta), 1e-6, 1-1e-6)
    term = (phi(delta)**2) / (Phi_delta * (1 - Phi_delta))
    ig = np.log(1 + (1.0/(s_lr**2)) * term * np.dot(v_diff, np.dot(Sigma, v_diff)))
    return ig


### Module 5: Heuristic Candidate Filters

In [5]:
def popularity_filter(candidate_ids, movie_popularity, pop_threshold):
    """Keep only movies with a rating count above the threshold."""
    return [mid for mid in candidate_ids if movie_popularity.get(mid, 0) >= pop_threshold]

def like_filter(candidate_ids, user_vector, movie_factors, movie_ids, topk):
    """
    Keep only the top-k movies predicted to be liked by the user.
    Predicted score is computed as the dot product between the movie factor and user_vector.
    """
    scores = {}
    for mid in candidate_ids:
        idx = movie_ids.index(mid)
        scores[mid] = np.dot(movie_factors[idx], user_vector)
    sorted_candidates = sorted(candidate_ids, key=lambda m: scores[m], reverse=True)
    return sorted_candidates[:topk]

def used_filter(candidate_ids, used_ids):
    """Remove movies that have been used in previous comparisons."""
    return [mid for mid in candidate_ids if mid not in used_ids]


### Module 6: EP Update (Surrogate via Local Laplace Approximation)

In [6]:
from numpy.linalg import inv

def ep_update(mu, Sigma, v_l, v_r, b_il, b_ir, response):
    """
    Perform one surrogate EP update.
    
    response = 1 if the user prefers movie l over r; else 0.
    
    Compute:
      delta = (b_il - b_ir + (v_l - v_r)^T mu) / s,   where s = sqrt(v_l^T Sigma v_l + v_r^T Sigma v_r)
    Then:
      For response==1, use:
         g = (phi(delta)/Phi(delta))*(x/s)
         lambda = (phi(delta)/Phi(delta))*((phi(delta)/Phi(delta)) + delta)/(s^2)
      For response==0, flip the sign.
      
    Update:
      mu_new = mu + Sigma @ g
      Sigma_new = inv(inv(Sigma) + lambda * np.outer(x, x))
    """
    delta, s, x = compute_delta(mu, Sigma, v_l, v_r, b_il, b_ir)
    if response == 0:
        delta = -delta  # flip for the opposite preference

    psi = phi(delta)
    Phi_val = np.clip(Phi(delta), 1e-6, 1-1e-6)
    
    if response == 1:
        g = (psi / Phi_val) * (x / s)
        lambda_val = (psi / Phi_val) * ((psi / Phi_val) + delta) / (s**2)
    else:
        g = -(psi / (1 - Phi_val)) * (x / s)
        lambda_val = (psi / (1 - Phi_val)) * ((psi / (1 - Phi_val)) - delta) / (s**2)
    
    mu_new = mu + Sigma @ g
    Sigma_inv = inv(Sigma)
    Sigma_new = inv(Sigma_inv + lambda_val * np.outer(x, x))
    return mu_new, Sigma_new


### Module 7: Static Adaptive Pairwise Feedback Simulation

In [7]:
def static_pairwise_update(user_id, user_item_matrix, movie_factors, movie_biases,
                           user_factors, movie_ids, movie_popularity,
                           num_queries=1, pop_threshold=1000, topk=50):
    """
    Perform a static simulation of the adaptive pairwise feedback procedure.
    
    Instead of an interactive loop, we use the static ratings data to
    simulate a fixed number (num_queries) of pairwise updates.
    
    Returns the final estimated user latent vector (posterior mean) and feedback history.
    """
    # Initialize user posterior using the factor model (using sample mean and covariance)
    mu0 = user_factors.mean(axis=0)
    Sigma0 = np.cov(user_factors.T) + 1e-6 * np.eye(user_factors.shape[1])
    mu, Sigma = mu0.copy(), Sigma0.copy()
    
    # Use movies that the user has rated (nonzero entries)
    user_ratings = user_item_matrix.loc[user_id]
    candidate_ids = user_ratings[user_ratings > 0].index.tolist()
    used_ids = set()
    feedback_history = []
    
    for q in range(num_queries):
        # Apply candidate filters
        filtered_ids = popularity_filter(candidate_ids, movie_popularity, pop_threshold)
        filtered_ids = like_filter(filtered_ids, mu, movie_factors, movie_ids, topk)
        filtered_ids = used_filter(filtered_ids, used_ids)
        
        if len(filtered_ids) < 2:
            print(f"Not enough candidates after filtering at query {q+1}")
            break
        
        # For simplicity, evaluate IG for adjacent pairs in sorted order
        scores = {}
        for mid in filtered_ids:
            idx = movie_ids.index(mid)
            scores[mid] = movie_biases[mid] + np.dot(movie_factors[idx], mu)
        
        sorted_ids = sorted(filtered_ids, key=lambda m: scores[m])
        best_ig = -np.inf
        best_pair = None
        for i in range(len(sorted_ids)-1):
            m1 = sorted_ids[i]
            m2 = sorted_ids[i+1]
            idx1 = movie_ids.index(m1)
            idx2 = movie_ids.index(m2)
            ig = information_gain(mu, Sigma, movie_factors[idx1], movie_factors[idx2],
                                  movie_biases[m1], movie_biases[m2])
            if ig > best_ig:
                best_ig = ig
                best_pair = (m1, m2)
        
        if best_pair is None:
            break
        
        movie_A, movie_B = best_pair
        used_ids.update(best_pair)
        
        # Simulate the user response from the static ratings:
        true_rating_A = user_item_matrix.loc[user_id, movie_A]
        true_rating_B = user_item_matrix.loc[user_id, movie_B]
        response = 1 if true_rating_A >= true_rating_B else 0
        
        feedback_history.append({
            'query': q+1,
            'movie_A': movie_A,
            'movie_B': movie_B,
            'true_rating_A': true_rating_A,
            'true_rating_B': true_rating_B,
            'response': response,
            'IG': best_ig
        })
        print(f"Static Query {q+1}: {movie_A} vs. {movie_B} -> Response: {'A' if response==1 else 'B'} (IG={best_ig:.4f})")
        
        # Update the user latent vector using the EP update module.
        idx_A = movie_ids.index(movie_A)
        idx_B = movie_ids.index(movie_B)
        mu, Sigma = ep_update(mu, Sigma,
                              movie_factors[idx_A], movie_factors[idx_B],
                              movie_biases[movie_A], movie_biases[movie_B],
                              response)
        
        # Optionally relax the popularity threshold after each query.
        pop_threshold = max(0, pop_threshold - 50)
    
    return mu, Sigma, feedback_history


### Module 8: Recommendation Generation

In [8]:
def generate_recommendations(mu, movie_factors, movie_biases, movie_ids, top_n=10):
    scores = []
    for mid in movie_ids:
        idx = movie_ids.index(mid)
        # Predicted rating: movie bias + dot(user latent, movie latent)
        score = movie_biases[mid] + np.dot(movie_factors[idx], mu)
        scores.append((mid, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    recommended = [mid for mid, score in scores[:top_n]]
    return recommended


In [9]:
import random

movies_file = 'data/Netflix_Dataset_Movie.csv'
ratings_file = 'data/Netflix_Dataset_Rating.csv'

# Module 1: Load data
movies, ratings, user_item_matrix, sparse_user_item = load_data(movies_file, ratings_file)

# Module 2: Train factor model and compute bias
user_factors, movie_factors, svd_model = train_factor_model(sparse_user_item, n_components=30)
global_bias, movie_biases_series, user_biases_series = compute_biases(user_item_matrix)
movie_biases = {mid: movie_biases_series[mid] for mid in movie_biases_series.index.tolist()}
movie_ids = user_item_matrix.columns.tolist()

movie_popularity = ratings.groupby('Movie_ID').size().to_dict()

available_user_ids = user_item_matrix.index.tolist()
user_id = random.choice(available_user_ids)
print("Selected User ID:", user_id)

# Module 7: Run a static pairwise feedback simulation
final_mu, final_Sigma, feedback_history = static_pairwise_update(
    user_id, user_item_matrix, movie_factors, movie_biases,
    user_factors, movie_ids, movie_popularity,
    num_queries=5,   
    pop_threshold=1000,
    topk=50
)

# Module 8: Generate recommendations with the updated latent vector
recommended_ids = generate_recommendations(final_mu, movie_factors, movie_biases, movie_ids, top_n=5)

recommended_movies = movies[movies['Movie_ID'].isin(recommended_ids)]
recommended_movies = recommended_movies.set_index('Movie_ID').loc[recommended_ids]
recommended_names = recommended_movies['Name'].tolist()

print("\nFinal estimated user latent vector (mu):")
print(final_mu)
print("\nFeedback History:")
for fb in feedback_history:
    print(fb)
print("\nTop recommended movies for user", user_id, ":", recommended_names)


Selected User ID: 386900
Static Query 1: 241 vs. 4266 -> Response: A (IG=0.5602)
Static Query 2: 2612 vs. 3150 -> Response: A (IG=0.5136)
Static Query 3: 1466 vs. 197 -> Response: B (IG=0.4855)
Static Query 4: 501 vs. 4402 -> Response: A (IG=0.4681)
Static Query 5: 2395 vs. 32 -> Response: A (IG=0.4856)

Final estimated user latent vector (mu):
[25.26709556 -0.79209306  0.71748474 -0.88099476 -1.46448787 -1.40831392
  0.0342057   0.77313304 -1.19099873 -0.18078485 -1.49405127  0.74528173
  1.40268965 -0.26208274 -0.66413475 -0.20076085  0.57038783  0.13262089
 -0.12291194  1.18911257  0.59295636  0.62111004 -0.3628359  -0.87968049
 -0.68870719  0.54098492 -0.72611832  0.67556544  0.25877973 -0.17676838]

Feedback History:
{'query': 1, 'movie_A': 241, 'movie_B': 4266, 'true_rating_A': np.float64(5.0), 'true_rating_B': np.float64(5.0), 'response': 1, 'IG': np.float64(0.5602033091256166)}
{'query': 2, 'movie_A': 2612, 'movie_B': 3150, 'true_rating_A': np.float64(2.0), 'true_rating_B': np.